# MP 4: Calibration and RANSAC

**Due date**: April 4, 2021 at 9:45am.

**Instructions**: Read and complete the problems below. In this assignment, you should be switched over to a local install. 

To submit your assignment, perform the following:

1. Double-check that your programs run without error.
2. Send this file, all of your .py files and any other files you used in your programs on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).  Do not rename the files.
3. If you are using any external libraries other than the ones that are indicated during the installation process, include a README file indicating which library and version you are using.  To be on the safe side, you should include a backup procedure until the graders verify that they are able to support use of that library.

## Problem 1: Estimation and sensitivity

In this problem, a simple dataset with 3D input *x* and 2D output *y* is given to you.  You will investigate the behavior of two model classes, an affine model and a quadratic model.

* The affine model states that $y = Ax+b $ where $\theta=stack(b,A)$ are the 8 model parameters stacked into a single vector (e.g., $\theta = (b_1, b_2, a_{11}, a_{12}, a_{13}, a_{21}, a_{22}, a_{23})$).
* The quadratic model states that $$y_i = \theta_{i0} + \theta_{i1}x_1 + \theta_{i2}x_2 + \theta_{i3}x_3 +  \theta_{i11}x_1^2 + \theta_{i12}x_1 x_2 + \theta_{i13}x_1 x_3 + \theta_{i22}x_2^2 + \theta_{i23}x_2 x_3 + \theta_{i33} x_3^2$$ for $i=1,2$. The first 4 parameters in this set of coefficients also exist in the affine model, but the remaining 6 multiply the order-2 monomials $x_1^2$, $x_1 x_2$, $x_1 x_3$, $x_2^2$, $x_2 x_3$, and $x_3^2$. The 20-parameter vector is then $\theta=(\theta_{10},\theta_{20},\theta_{11},...,\theta_{13},\theta_{21},...,\theta_{23},\theta_{111},...,\theta_{133},\theta_{211},...,\theta_{233})$.

### Problem 1.A

For a given model, the loss function measures the error between a prediction $f(x;\theta)$ and the output $y$ for a single datapoint $(x,y)$.  Implement the models and the squared L2 loss function $\|y-f(x;\theta)\|$ for each model in the cell below.

In [9]:
import numpy as np
import scipy
import scipy.optimize

############################# Problem 1.A code goes here ############################

def affine_model(x,theta):
    #implement the first model here
    theta = np.array(theta)
    b = theta[:2]
    A = theta[2:]
    A = np.reshape(A, (2,3))
    y = (A@x+b).flatten()
    return [y[0], y[1]]

def quadratic_model(x,theta):
    #implement the second model here
    theta10, theta20, theta11, theta12, theta13, theta21, theta22, theta23,\
    theta111, theta112, theta113, theta122, theta123, theta133,\
    theta211, theta212, theta213, theta222, theta223, theta233 = theta
    
    x1, x2, x3 = x
    
    y1 = theta10 + theta11*x1 + theta12*x2 + theta13*x3 + theta111*x1**2 + theta112*x1*x2 + theta113*x1*x3 + theta122*x2**2\
        + theta123*x2*x3 + theta133*x3**2
    y2 = theta20 + theta21*x1 + theta22*x2 + theta23*x3 + theta211*x1**2 + theta212*x1*x2 + theta213*x1*x3 + theta222*x2**2\
        + theta223*x2*x3 + theta233*x3**2
    return [y1,y2]

def affine_squared_L2_loss(y,x,theta):
    #implement the loss function here
    f = affine_model(x, theta)
    res = np.linalg.norm(np.array(y) - np.array(f))
    return res

def quadratic_squared_L2_loss(y,x,theta):
    #implement the loss function here
    f = quadratic_model(x, theta)
    res = np.linalg.norm(np.array(y) - np.array(f))
    return res

#####################################################################################,

**Written.**  In the quadratic model, why did we omit parameters $\theta_{i21}$, $\theta_{i31}$, and $\theta_{i32}$ which would act as coefficients of the monomials $x_2 x_1$, $x_3 x_1$, and $x_3 x_2$, respectively?

### Problem 1.B

To estimate the parameters, you will construct objective functions `affine_objective` and `quadratic_objective` that sum the loss over the provided dataset.  To minimize the objective function, use the [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) function.  I suggest using the 'BFGS' method and no more than 30 or 50 iterations.  Note that you will either need to write your objective function as a 1-parameter wrapper function over the optimization parameters `theta`, or to use the `args` argument to `minimize` to pass in the dataset.  Your code that calls `minimize` should input an initial guess `x0=theta0` and extract the results from the returned `OptimizeResult`.

Note: on the ground truth optimization, your optimizer for the quadratic model should return an objective value (`OptimizeResult.fun`) close to zero.

**Written.** Report the estimated values for each model and their errors.  Compare the magnitudes of the affine model and their corresponding terms in the quadratic model.  Offer an explanation of what you observe.

In [16]:
############################# Problem 1.B code goes here ############################

def affine_objective(theta,dataset):
    """
    Args:
        theta: the parameter vector
        dataset: a list of pairs (x,y)
    """
    #implement the objective function
    total_loss = 0
    for data in dataset:
        x, y = data
        total_loss += affine_squared_L2_loss(y,x,theta)
    return total_loss

def quadratic_objective(theta,dataset):
    """
    Args:
        theta: the parameter vector
        dataset: a list of pairs (x,y)
    """
    #implement the objective function
    total_loss = 0
    for data in dataset:
        x, y = data
        total_loss += quadratic_squared_L2_loss(y,x,theta)
    return total_loss

def optimize_affine(dataset):
    initial_guess = np.array([0]*8)
    res = scipy.optimize.minimize(affine_objective, initial_guess, args=(dataset,), options={"maxiter":50})
    return res.x

def optimize_quadratic(dataset):
    initial_guess = np.array([0]*20)
    res = scipy.optimize.minimize(quadratic_objective, initial_guess, args=(dataset,), options={"maxiter":50})
    return res.x

#####################################################################################

In [17]:
#Driver code

dataset_x = np.load('fitting/dataset_x.npy')
dataset_y = np.load('fitting/dataset_y.npy')
dataset = list(zip(dataset_x,dataset_y))

affine_parameters = optimize_affine(dataset)
quadratic_parameters = optimize_quadratic(dataset)
print("Affine:",' '.join(["%.3f"%v for v in affine_parameters]))
print("Quadratic:",' '.join(["%.3f"%v for v in quadratic_parameters]))

dataset_y_ground_truth = np.load('fitting/dataset_y_ground_truth.npy')
dataset_ground_truth = list(zip(dataset_x,dataset_y_ground_truth))

affine_parameters_ground_truth = optimize_affine(dataset_ground_truth)
quadratic_parameters_ground_truth = optimize_quadratic(dataset_ground_truth)

print("Affine ground truth:",' '.join(["%.3f"%v for v in affine_parameters_ground_truth]))
print("Quadratic ground truth:",' '.join(["%.3f"%v for v in quadratic_parameters_ground_truth]))

print("Affine errors:",' '.join(["%.3f"%abs(v) for v in affine_parameters-affine_parameters_ground_truth]))
print("Quadratic errors:",' '.join(["%.3f"%abs(v) for v in quadratic_parameters-quadratic_parameters_ground_truth]))

Affine: 10.504 60.300 0.268 -0.011 0.177 0.003 -0.373 1.253
Quadratic: 10.517 60.345 0.149 0.132 0.069 0.088 -0.434 1.070 0.131 0.010 -0.051 -0.135 -0.014 0.134 -0.209 -0.067 0.336 0.128 0.006 -0.006
Affine ground truth: 10.474 60.283 0.303 0.020 0.199 0.002 -0.401 1.302
Quadratic ground truth: 10.502 60.298 0.098 0.001 0.296 0.003 -0.600 1.405 0.200 0.003 -0.000 -0.003 0.002 -0.098 -0.002 0.002 -0.002 0.198 0.002 -0.104
Affine errors: 0.030 0.017 0.035 0.030 0.022 0.001 0.028 0.049
Quadratic errors: 0.015 0.048 0.051 0.132 0.228 0.084 0.166 0.335 0.069 0.007 0.051 0.132 0.016 0.233 0.207 0.069 0.339 0.070 0.004 0.097


### Problem 1.C. Errors and sensitivity

To estimate how sensitive the parameter estimates are to noise, we will use the inverse of the hessian of the objective function.  Use the approximation $var(\theta) \approx O(diag(H^{-1})/n)$ with $H$ the hessian of the sum-of-squared errors objective at $\theta$ and $n$ the number of samples in the dataset.  You should return the *standard error* of the parameter estimate, which is the vector of square roots of the variances.  (Remark: this approximation is based on the Cramer-Rao lower bound in statistics.)

To approximate the Hessian, you should use the *finite difference* approximation $$H_{ij} \approx (f(\theta + e_i h + e_j h) - f(\theta + e_i h) - f(\theta + e_j h) + f(\theta))/h^2$$ where $e_i$ is the elementary basis vector (i.e., the vector of all 0's except a 1 in the i'th entry) and *h* is a step size parameter.


In [36]:
############################# Problem 1.C code goes here ############################

def computeHessian(f, theta, dataset, h=0.0001):
    theta = np.array(theta)
    hessian_size = theta.shape[0]
    hessian = np.zeros((hessian_size,hessian_size))
    for i in range(hessian_size):
        for j in range(hessian_size):
            ei = np.array([0]*hessian_size)
            ei[i] = 1
            ej = np.array([0]*hessian_size)
            ej[j] = 1
            hessian[i,j] = (f(theta+ei*h+ej*h,dataset)-f(theta+ei*h,dataset)-f(theta+ej*h,dataset)+f(theta,dataset))/(h**2)
    return hessian

def affine_sensitivity(theta,dataset):
    # Compute Hessian
    H = computeHessian(affine_objective, theta, dataset)
    var = np.diag(np.linalg.inv(H))/len(dataset)
    print(var)
    return np.sqrt(var)

def quadratic_sensitivity(theta,dataset):
    H = computeHessian(quadratic_objective, theta, dataset)
    var = np.diag(np.linalg.inv(H))/len(dataset)
    print(var)
    return np.sqrt(var)

#####################################################################################

The code below will run your sensitivity estimation code for the normal 100-example dataset as well as a 20-example subset.  

**Written.** Why are parameters estimated to be more sensitive for the quadratic model? Why are parameters estimated to be more sensitive when the dataset is small?  Do the estimates correspond in magnitude to the empirical errors?

What happens to the estimated sensitivies when the dataset size drops below 10?  Why?

In [37]:

print("Affine sensitivity:\t",' '.join(["%.3f"%v for v in affine_sensitivity(affine_parameters,dataset)]))
print("Affine errors:\t\t",' '.join(["%.3f"%abs(v) for v in affine_parameters-affine_parameters_ground_truth]))
print("Quadratic sensitivity:\t",' '.join(["%.3f"%v for v in quadratic_sensitivity(quadratic_parameters,dataset)]))
print("Quadratic errors:\t",' '.join(["%.3f"%abs(v) for v in quadratic_parameters-quadratic_parameters_ground_truth]))

dataset_small = dataset[:20]
affine_parameters_small = optimize_affine(dataset_small)
quadratic_parameters_small = optimize_quadratic(dataset_small)
print()
print("Affine sensitivity, small dataset:\t",' '.join(["%.3f"%v for v in affine_sensitivity(affine_parameters_small,dataset_small)]))
print("Affine errors, small dataset:\t\t",' '.join(["%.3f"%abs(v) for v in affine_parameters_small-affine_parameters_ground_truth]))
print("Quadratic sensitivity, small dataset:\t",' '.join(["%.3f"%v for v in quadratic_sensitivity(quadratic_parameters_small,dataset_small)]))
print("Quadratic errors, small dataset:\t",' '.join(["%.3f"%abs(v) for v in quadratic_parameters_small-quadratic_parameters_ground_truth]))


[0.00014863 0.00020025 0.00017527 0.00016881 0.00018968 0.00021959
 0.00025989 0.00025185]
Affine sensitivity:	 0.012 0.014 0.013 0.013 0.014 0.015 0.016 0.016
Affine errors:		 0.030 0.017 0.035 0.030 0.022 0.001 0.028 0.049
[ 0.0006569   0.00042729  0.0037505   0.00059484 -0.00040279  0.00244983
  0.00036189 -0.00037928  0.00141709  0.00055494  0.00041492 -0.00011571
 -0.00025553 -0.00019672  0.0009222   0.0003115   0.00021737 -0.00022349
 -0.0006296  -0.0005147 ]
Quadratic sensitivity:	 0.026 0.021 0.061 0.024 nan 0.049 0.019 nan 0.038 0.024 0.020 nan nan nan 0.030 0.018 0.015 nan nan nan
Quadratic errors:	 0.015 0.048 0.051 0.132 0.228 0.084 0.166 0.335 0.069 0.007 0.051 0.132 0.016 0.233 0.207 0.069 0.339 0.070 0.004 0.097


<ipython-input-36-fd89faae663a>:27: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(var)



[-1.63367904e-06 -1.93511466e-06 -4.44474980e-06 -2.63927443e-05
 -1.82864809e-05 -6.02693233e-06 -3.88500081e-05 -3.27663131e-05]
Affine sensitivity, small dataset:	 nan nan nan nan nan nan nan nan
Affine errors, small dataset:		 0.003 0.044 0.012 0.003 0.039 0.007 0.028 0.051


<ipython-input-36-fd89faae663a>:21: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(var)


[ 1.07319075  1.3557496   4.23626355  1.50857111  0.02689101  0.3371512
  3.22351151 15.51265549  1.46013793 -5.45977175  6.00909524 -2.47269964
  2.42224568 -1.68021586 -0.9170329  -2.64753613 -2.01144454 -3.39962813
  2.39909591  2.57633083]
Quadratic sensitivity, small dataset:	 1.036 1.164 2.058 1.228 0.164 0.581 1.795 3.939 1.208 nan 2.451 nan 1.556 nan nan nan nan nan 1.549 1.605
Quadratic errors, small dataset:	 0.008 1.011 0.035 0.290 0.066 2.078 0.053 1.819 0.009 0.098 0.275 0.193 0.083 0.054 1.160 0.350 1.146 0.498 0.438 0.784


<ipython-input-36-fd89faae663a>:27: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(var)


## Problem 2: Plane extraction

We're going to perform intrinsics calibration of the SR300 depth camera by pointing it at perpendicular planes.  The first step is to extract the points that make up these planes.  To do so, in this problem we will implement a RANSAC-based plane extractor.  To keep your algorithms running at a reasonable speed, it will be helpful for you to learn array addressing in Numpy; running for loops in Python code is much, much slower.

The calibration color and depth images are stored in the `calibration/` folder, and they will be automatically loaded for you.  Specifically, we will be using the `depth_aligned_X.png` files.

**Note**: Points with coordinates (0,0,0) are *invalid points* that do not correspond to any depth reading.  Your code should ignore these points when trying to extract planes. 

### Problem 2.A

Implement a RANSAC algorithm under `extract_planes_ransac_a` in `planes.py`.  For `N` iterations, sample subsets of `m` points, fit planes, and examine how many inliers there are.  An inlier to a plane `(a,b,c,d)` is defined by $|(a,b,c)^T \mathbf{p} + d| < $ `inlier_tolerance`.

In this part, you will produce a naive implementation that outputs any inlier sets that are at least as large as `inlier_count`.   You will represent an inlier set using just a list of integers; the result from this function is a list of lists of integers.  The `fit_plane3` and `fit_plane` subroutines will allow you to fit these planes easily.

Running `planes.py` will perform this fitting for each of the depth images; the result will look something like this:

![Example output](example_output/planes_2a.png)


### Problem 2.B

The naive extractor obviously has problems, the main one being that the inlier sets overlap significantly.  Implement a refined version in `extract_planes_ransac_b` that 

1. Does not sample from candidate points that already belong to an extracted plane,
2. After the main RANSAC iteration, assigns points to the largest plane (in terms of # of inliers) for which they are an inlier, and 
3. Discards planes that have fewer than `inlier_count` points assigned to them, after step 2.

After uncommenting `PROBLEM='2b'`, the result should be much better, looking something like this:

![Example output](example_output/planes_2b.png)



### Problem 2.C

There are still some small errors near the boundary of the plane segmentation, which become more obvious when the inlier threshold is set larger.  In `extract_planes_ransac_c`, if a point can be an inlier for multiple planes, assign it to the one for which it is the best fit (i.e., the point-to-plane distance is lowest).  You might also modify the code to provide normals to your code, which you can use for even better assignments. 

After uncommenting `PROBLEM='2c'`, examine the boundaries between planes to ensure that you're getting reasonable results.   Tune the parameters to yield results that are as good as possible -- you might also want the parameters to depend on the scale of the input.  (Scan 9 is particularly challenging).

![Example output](example_output/planes_2c.png)

Then, for problem 3, click through each of the scans (or comment out the `vis.dialog()` call) to output the sets of planes for each image (the `planesets` variable).  This will output a file called `planesets.json` which you will include in your submission.

## Problem 3: Intrinsic calibration

Now, we will assume that we are given a default calibration for the camera taken from the manufacturer's specs, and use optimization to calibrate the $f_x, f_y, c_x, $ and $c_y$ intrinsic parameters.

### Problem 3.A

In `depth_calibration.py`, implement the `mutual_orthogonality()` function to determine how close to orthogonal are the planes determined for each scan.  You will keep the point indices belonging to each plane (the `planesets` variable) the same regardless of the setting of `fx,fy,cx,cy`, but you will change the points themselves.  In particular, see lines 165-169 of `rgbd.py` to see how the points are affected by these parameters. 

To measure orthogonality may choose to use an absolute value of the dot product (i.e., $|\cos \theta|$), dot product squared, angular error ($|90^\circ - \theta|$), or angular error squared.  To aggregate, you can either sum or average.  In any case, the result should be minimized when the normals of all planes detected for a scan are perfectly at 90 degrees.  (Note that for scan 11, two of the planes are parallel -- you should not penalize them.)

In the space below for written answers, describe the orthogonality metric you used and the method you used for aggregation.  Also, examine which of the scans produces the worst result(s), and check the visualization in `planes.py`.  Explain why those particular scans seemed problematic.

### Problem 3.B

Now, implement the `calibrate_intrinics_fxfy` function to optimize your metric only over the `fx,fy` variables, keeping `cx,cy` fixed.  Use the [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) function to minimize an objective function.  I suggest using the 'Nelder-Mead' method, which is a derivative-free method, and no more than 30 or 50 iterations.  Note that you will need to write your objective function as a 1-parameter wrapper function such that it accepts the optimization parameters as an array `x`.  Your code that calls `minimize` should input an initial guess `x0=(fx,fy)` and extract the results from the returned array.

In the space below for written answers, report the initial and final parameter values, and the initial and final objective function values.  Did this seem to converge to reasonable values?  Compare with the parameters set in `calibration/color_intrinsics.json` (note: we're not using `depth_intrinsics.json` because the depth images have been aligned to the color camera).

### Problem 3.C

Now, implement the `calibrate_intrinics_all` function to optimize your metric over all four variables.   In the space below for written answers, report the initial and final parameter values, and the initial and final objective function values.  Did this seem to converge to reasonable values compared to `calibration/color_intrinsics.json`?  (Note that the Intel Realsense library refers to `cx, cy` as `ppx, ppy`.)  Explain why or why not.

Is this problem identifiable?  Give a rough justification for your answer (i.e., by verbally describing the geometry of the situation).

### Problem 3.D (IR2 section only)

Provide a rigorous mathematical justification for whether the full optimization in this problem is identifiable or not. 

## Written responses

### Written response for Problem 1.A

Put your answer here.

### Written response for Problem 1.B

Put your answer here.

### Written response for Problem 1.C

Put your answer here.

### Written response for Problem 3.A

Put your answer here.

### Written response for Problem 3.B

Put your answer here.

### Written response for Problem 3.C

Put your answer here.


### Written response for Problem 3.D (IR2 section only)

Put your answer here.